In [ ]:
# train_model.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib

# === Load your CSV with labelled examples (defaulted: 1, not_default: 0)
##df = pd.read_csv("loan_data.csv")  # expects columns described earlier + 'defaulted'

# Simple feature engineering
##df['debt_to_income'] = df['existing_debt'] / (df['annual_income'] + 1e-6)
##features = ['age', 'annual_income', 'credit_score', 'debt_to_income', 'loan_amount', 'loan_term_months']
##X = df[features].fillna(0)
##y = df['defaulted']
df = pd.read_csv("/content/Loan_Modelling.csv")
# Drop ID and ZIPCode since they don’t affect loan decisions
X = df.drop(columns=["ID", "ZIPCode", "Personal_Loan"])
y = df["Personal_Loan"]
featurs=X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier(n_estimators=200, max_depth=4, random_state=42))
])
pipe.fit(X_train, y_train)

# Eval
y_prob = pipe.predict_proba(X_test)[:,1]
y_pred = (y_prob > 0.5).astype(int)
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred, digits=4))

# Save model + feature list
joblib.dump({'model': pipe, 'features': featurs}, "loan_model_v1.joblib")

ROC AUC: 0.9992855825958702
              precision    recall  f1-score   support

           0     0.9967    0.9956    0.9961       904
           1     0.9588    0.9688    0.9637        96

    accuracy                         0.9930      1000
   macro avg     0.9777    0.9822    0.9799      1000
weighted avg     0.9930    0.9930    0.9930      1000



['loan_model_v1.joblib']

In [ ]:
import joblib
import pandas as pd

# Load the saved model pipeline and feature list
loaded_model_info = joblib.load("loan_model_v1.joblib")
loaded_pipe = loaded_model_info['model']
features = loaded_model_info['features']

# Create a DataFrame for the new person with the correct feature names
# The order and names of features must match the training data
new_person_data = [[35, 10, 80000, 3, 2.5, 2, 0, 0, 1, 1, 1]]
new_person_df = pd.DataFrame(new_person_data, columns=features)

# Make prediction using the loaded pipeline (scaling is handled internally)
prediction = loaded_pipe.predict(new_person_df)

print("Loan Eligibility:", "Approved" if prediction[0] == 1 else "Not Approved")

Loan Eligibility: Approved


In [ ]:
# app_agent.py
from flask import Flask, request, jsonify
import joblib
import numpy as np
import os

app = Flask(__name__)
store = joblib.load("loan_model_v1.joblib")
model = store['model']
features = store['features']

# Example business policy thresholds:
MAX_LOAN_BY_INCOME_RATIO = 0.5  # loan_amount <= 0.5 * annual_income
AUTO_REJECT_CREDIT_SCORE = 350
AUTO_ACCEPT_CREDIT_SCORE = 800

def validate_input(data):
    required = ['age','annual_income','credit_score','existing_debt','loan_amount','loan_term_months']
    for r in required:
        if r not in data:
            return False, f"Missing field: {r}"
    return True, ""

def apply_business_rules(data):
    if data['credit_score'] <= AUTO_REJECT_CREDIT_SCORE:
        return 'decline', 'credit_score_below_minimum'
    if data['credit_score'] >= AUTO_ACCEPT_CREDIT_SCORE and data['existing_debt'] == 0:
        return 'approve', 'exceptionally_high_credit_score_and_no_debt'
    ratio = data['loan_amount'] / (data['annual_income'] + 1e-6)
    if ratio > MAX_LOAN_BY_INCOME_RATIO:
        return 'decline', 'loan_too_large_for_income'
    return None, None

@app.route('/check', methods=['POST'])
def check():
    data = request.json
    ok, msg = validate_input(data)
    if not ok:
        return jsonify({'error': msg}), 400

    # business rules
    rule_decision, rule_reason = apply_business_rules(data)
    if rule_decision:
        return jsonify({'decision': rule_decision, 'reason': rule_reason, 'score': None})

    # prepare features for model
    x = np.zeros((1, len(features)))
    prepared = {
        'age': data['age'],
        'annual_income': data['annual_income'],
        'credit_score': data['credit_score'],
        'debt_to_income': data['existing_debt'] / (data['annual_income'] + 1e-6),
        'loan_amount': data['loan_amount'],
        'loan_term_months': data['loan_term_months']
    }
    x[0] = [prepared[f] for f in features]
    score = float(model.predict_proba(x)[0,1])  # probability of default

    # decision thresholds (policy): if default_prob < 0.1 approve, >0.4 decline, otherwise manual review
    if score < 0.1:
        decision = 'approve'
    elif score > 0.4:
        decision = 'decline'
    else:
        decision = 'manual_review'

    # simple explainability: top features by absolute z-score
    # (for production use SHAP or LIME)
    diffs = np.abs((x - model.named_steps['scaler'].mean_) / (model.named_steps['scaler'].scale_ + 1e-9))
    top_idx = np.argsort(-diffs[0])[:3]
    reasons = [f"{features[i]} is {x[0,i]:.2f}" for i in top_idx]

    return jsonify({'decision': decision, 'score_default_prob': score, 'top_reasons': reasons})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
